In [1]:
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt
from skimage.io import imread, imshow,imsave
from skimage.color import rgb2hsv
from PIL import Image
from skimage.util import img_as_ubyte
import json
from skimage import measure 
from shapely.geometry import Polygon, MultiPolygon
import time

In [2]:
obj = {
    "inf0":{
        "description": "Weed Detection"
    },
    "images": [],
    "annotations": [],
    "categories": [
        {
            "id": 1,
            "name": "Brinjal"
        },
        {
            "id": 2,
            "name": "Broad_Beans"
        }
    ]
}

In [3]:
folder_dir = "D:\\Final Yr Project\\shadow_cleared_data"
imgid = 1
imgs = []
for fold in os.listdir(folder_dir):
    for images in os.listdir(folder_dir+"\\"+fold):
        if (images.endswith(".jpg")):
            temp = {
                "id": imgid,
                "width": 512,
                "height": 512,
                "file_name": images
            }
            imgs.append(temp)
            imgid+=1
obj['images'] = imgs

In [8]:
def func1(path,images):
    global aid,cid,iid,annotations
    #img2 = cv2.imread(path, cv2.IMREAD_COLOR)
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    _,threshold = cv2.threshold(img, 15, 75, 
                            cv2.THRESH_BINARY)

    # Detecting shapes in image by selecting region 
    # with same colors or intensity.
    contours,_=cv2.findContours(threshold, cv2.RETR_TREE,
                            cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours :
        area = cv2.contourArea(cnt)
        if area > 10000:
            approx = cv2.approxPolyDP(cnt, 0.0005 * cv2.arcLength(cnt, True), True)
            x,y,w,h = cv2.boundingRect(cnt)
            temp1 = {
                "id": aid,
                "iscrowd": 0,
                "image_id": iid,
                "category_id": cid,
                "segmentation": [],
                "bbox": [],
                "area": 0
            }
            seg = []
            for i in range(len(approx)):
                for x1,y1 in approx[i]:
                    seg.append(float(x1))
                    seg.append(float(y1))
                    
            temp1["segmentation"] = [seg]
            temp1["bbox"] = [x,y,w,h]
            temp1["area"] = area
            aid+=1
            annotations.append(temp1)
    iid+=1
    aid+=1
            

In [9]:
def func(path,images):
    crop=cv2.imread(path)
    crop_hsv = rgb2hsv(crop)
    lower_mask = crop_hsv[:,:,0] > 0.1
    #refer to hue channel (in the colorbar)
    upper_mask = crop_hsv[:,:,0] < 0.45
    #refer to transparency channel (in the colorbar)
    saturation_mask = crop_hsv[:,:,1] > 0.3

    mask = upper_mask*lower_mask*saturation_mask
    red = crop[:,:,0]*mask
    green = crop[:,:,1]*mask
    blue = crop[:,:,2]*mask
    crop_masked = np.dstack((red,green,blue))
    imsave("D:\\Final Yr Project\\data\\test.jpg",crop_masked)
    func1("D:\\Final Yr Project\\data\\test.jpg",images)

In [10]:
folder_dir = "D:\\Final Yr Project\\shadow_cleared_data"
annotations = []
iid = 1
cid = 1
aid = 0
ct = 0
for fold in os.listdir(folder_dir):
    for images in os.listdir(folder_dir+"\\"+fold):
        if (images.endswith(".jpg")):
            path = "D:\\Final Yr Project\\shadow_cleared_data\\{}\\{}".format(fold,images)
            func(path,images)
            #print("No of Images : ",ct,end='\r')
            ct+=1
        else:
            print()
    print()
    cid+=1
obj["annotations"] = annotations

brinjal1545.jpg
brinjal1653.jpg
brinjal1797.jpg
brinjal2401.jpg
brinjal2413.jpg
brinjal2513.jpg
brinjal2909.jpg
brinjal3305.jpg
brinjal3961.jpg




In [43]:
obj['images'][123]

{'id': 124, 'width': 512, 'height': 512, 'file_name': 'brinjal1279.jpg'}

In [14]:
t = []
for x in annotations:
    t.append(x['image_id'])
tt = list(set(t))

for i in range(1,4739):
    if(i not in tt):
        print(i)

247
296
365
600
605
645
786
878
1031
1066
1191
1312
1346
2090
2229
2264
2266
2336
2450
2583
3232
3480
3569
3572
3743
3867
3932
4379
4619


In [17]:
json_object = json.dumps(obj, separators=(',', ':')) 
with open("shadow_crcted_coco.json", "w") as outfile:
    outfile.write(json_object)